In [1]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
import logging
from dotenv import load_dotenv
import os

# Load environment variables from env.txt (or .env if renamed)
load_dotenv(dotenv_path="env.txt")  # Use ".env" if that's the file name

# Logging setup
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s -%(message)s",
    filemode="a")

# Read database credentials from environment
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_DATABASE = os.getenv("PG_DATABASE")

# Encode password and database name for URL safety
PG_PASSWORD_ENCODED = urllib.parse.quote_plus(PG_PASSWORD)
PG_DATABASE_ENCODED = urllib.parse.quote_plus(PG_DATABASE)

# Create SQLAlchemy engine
engine = create_engine(
    f'postgresql+psycopg2://{PG_USER}:{PG_PASSWORD_ENCODED}@{PG_HOST}:{PG_PORT}/{PG_DATABASE_ENCODED}'
)

# SQL query to list all table names in the 'public' schema
tables_query = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public' 
  AND table_type = 'BASE TABLE';
"""

# Run query and load result in DataFrame
tables = pd.read_sql_query(tables_query, engine)

# Show tables
tables


,table_name
0,begin_inventory
1,end_inventory
2,purchases
3,purchase_prices
4,sales
5,vendor_invoice
6,vendor_sales_summary


In [6]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
import logging 
import time
import os

print ("code start")
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s -%(levelname)s - %(message)s",
    filemode="a")

def ingest_db(df,table_name,engine):
    df.to_sql(table_name,con=engine,schema="public",if_exists="replace",index=False)
    print(f"✅ Table '{table_name}' uploaded successfully to schema 'public'.")

def load_raw_data():
    start=time.time()
    for file in os.listdir('data'):
        if file.endswith('.csv'):
            file_path = os.path.join('data',file)
            df=pd.read_csv(file_path)
            logging.info(f"📦 Ingesting '{file}' with shape {df.shape}")
            table_name=file.split('.')[0].lower()
            ingest_db(df,table_name,engine)
    end = time.time()  # ✅ needs parentheses
    total_time = (end - start) / 60
    logging.info('--------------ingestion complete----------------')
    logging.info(f'Total time taken : {total_time} minutes')


if __name__ == '__main__':
    load_raw_data()





    

code start
✅ Table 'begin_inventory' uploaded successfully to schema 'public'.
✅ Table 'end_inventory' uploaded successfully to schema 'public'.
✅ Table 'purchases' uploaded successfully to schema 'public'.
✅ Table 'purchase_prices' uploaded successfully to schema 'public'.
✅ Table 'sales' uploaded successfully to schema 'public'.
✅ Table 'vendor_invoice' uploaded successfully to schema 'public'.


In [2]:
import os
print("Current working directory:",os.getcwd())

Current working directory: C:\Users\HP\Desktop\2nd Project


In [3]:
import pandas as pd
tables = pd.read_sql_query(tables_query, engine)
tables

,table_name
0,begin_inventory
1,end_inventory
2,purchases
3,purchase_prices
4,sales
5,vendor_invoice


In [4]:
for table in tables['table_name']:
    print('-'*50, f'{table}','-'*50)
    print('Count of Records:',pd.read_sql(f" select count(*)  from {table}",engine)['count'].values[0])
    pd.read_sql("select count(*) from purchases",engine)
    display(pd.read_sql(f"select * from {table} limit 5", engine))


-------------------------------------------------- begin_inventory --------------------------------------------------
Count of Records: 206529


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


-------------------------------------------------- end_inventory --------------------------------------------------
Count of Records: 224489


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


-------------------------------------------------- purchases --------------------------------------------------
Count of Records: 2372474


...

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


-------------------------------------------------- sales --------------------------------------------------
Count of Records: 12825363


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


-------------------------------------------------- vendor_invoice --------------------------------------------------
Count of Records: 5543


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


In [5]:
purchases=pd.read_sql_query('select * from purchases where "VendorNumber" =4466',engine)
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,31_HORNSEY_5255,31,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13180,2024-11-23,2024-11-28,2024-12-08,2025-01-15,9.35,6,56.10,1
2188,67_EANVERNESS_3140,67,3140,TGI Fridays Orange Dream,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13180,2024-11-23,2024-12-02,2024-12-08,2025-01-15,11.19,6,67.14,1
2189,73_DONCASTER_5255,73,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13180,2024-11-23,2024-12-01,2024-12-08,2025-01-15,9.35,6,56.10,1
2190,34_PITMERDEN_5255,34,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13180,2024-11-23,2024-11-28,2024-12-08,2025-01-15,9.35,6,56.10,1


In [6]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

,,Quantity,Dollars
Brand,PurchasePrice,,
3140,11.19,4640,51921.60
5215,9.41,4923,46325.43
5255,9.35,6215,58110.25


In [8]:
sales=pd.read_sql_query('select * from sales where "VendorNo" =4466',engine)
sales

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
1,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-12,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-15,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
3,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
4,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9448,9_BLACKPOOL_5215,9,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-12-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9449,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-02,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9450,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9451,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE


In [9]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()


,SalesDollars,SalesPrice,SalesQuantity
Brand,,,
3140,50531.10,30071.85,3890
5215,60416.49,41542.02,4651
5255,79187.04,51180.60,6096


In [10]:
vendor_invoice=pd.read_sql_query('select * from vendor_invoice limit 1 ',engine)
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None


In [11]:
begin_inventory=pd.read_sql_query('select * from begin_inventory limit 1 ',engine)
begin_inventory

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01


In [12]:
end_inventory=pd.read_sql_query('select * from begin_inventory limit 1 ',engine)
end_inventory

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01


In [13]:
purchase_prices=pd.read_sql_query('select * from purchase_prices limit 1 ',engine)
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD


In [5]:
freight_summary = pd.read_sql_query("""
    SELECT "VendorNumber", SUM("Freight") AS "Freightcost"
    FROM vendor_invoice
    GROUP BY "VendorNumber"
""", engine)
freight_summary
    

,VendorNumber,Freightcost
0,7255,648.71
1,1439,0.27
2,9751,24.53
3,6830,360.29
4,1587,6070.09
...,...,...
121,9625,1933.19
122,7154,432.37
123,1392,79528.99
124,1189,609.39


In [6]:
pd.read_sql_query("""
    SELECT 
        "VendorNo",
        "Brand",
        SUM("SalesDollars") AS TotalSalesDollars,
        SUM("SalesPrice") AS TotalPrice,
        SUM("SalesQuantity") AS TotalSalesQuantity,
        SUM("ExciseTax") AS TotalExciseTax
    FROM sales
    GROUP BY "VendorNo", "Brand"
""", engine)


,VendorNo,Brand,totalsalesdollars,totalprice,totalsalesquantity,totalexcisetax
0,2,90085,665.82,295.92,18.0,2.00
1,2,90609,599.76,449.82,24.0,0.52
2,60,771,704.53,494.67,47.0,37.01
3,60,3979,66871.69,41682.51,3931.0,7224.06
4,105,2529,359.88,59.98,12.0,9.44
...,...,...,...,...,...,...
11267,173357,2804,6298.60,3194.29,140.0,110.33
11268,173357,3666,8996.40,4873.05,360.0,141.19
11269,173357,3848,185.94,92.97,6.0,4.71
11270,173357,3909,24540.18,14469.21,982.0,773.87


In [7]:
pd.read_sql_query("""
    SELECT 
        "VendorNo",
        "Brand",
        SUM("SalesDollars") AS TotalSalesDollars,
        SUM("SalesPrice") AS TotalPrice,
        SUM("SalesQuantity") AS TotalSalesQuantity,
        SUM("ExciseTax") AS TotalExciseTax
    FROM sales
    GROUP BY "VendorNo", "Brand"
""", engine)


,VendorNo,Brand,totalsalesdollars,totalprice,totalsalesquantity,totalexcisetax
0,2,90085,665.82,295.92,18.0,2.00
1,2,90609,599.76,449.82,24.0,0.52
2,60,771,704.53,494.67,47.0,37.01
3,60,3979,66871.69,41682.51,3931.0,7224.06
4,105,2529,359.88,59.98,12.0,9.44
...,...,...,...,...,...,...
11267,173357,2804,6298.60,3194.29,140.0,110.33
11268,173357,3666,8996.40,4873.05,360.0,141.19
11269,173357,3848,185.94,92.97,6.0,4.71
11270,173357,3909,24540.18,14469.21,982.0,773.87


In [8]:
import time
start = time.time()

vendor_sales_summary = pd.read_sql_query("""
WITH FreightSummary AS (
    SELECT "VendorNumber",
           SUM("Freight") AS "FreightCost"
    FROM vendor_invoice
    GROUP BY "VendorNumber"
),
PurchaseSummary AS (
    SELECT 
        p."VendorNumber",
        p."VendorName",
        p."Brand",
        p."Description",
        p."PurchasePrice",
        pp."Price" AS "ActualPrice",
        pp."Volume",
        SUM(p."Quantity") AS "TotalPurchaseQuantity",
        SUM(p."Dollars") AS "TotalPurchaseDollars"
    FROM purchases p 
    JOIN purchase_prices pp
        ON p."Brand" = pp."Brand"
    WHERE p."PurchasePrice" > 0
    GROUP BY 
        p."VendorNumber", 
        p."VendorName", 
        p."Brand", 
        p."Description", 
        p."PurchasePrice", 
        pp."Price", 
        pp."Volume"
),


SalesSummary AS (
    SELECT 
        "VendorNo",
        "Brand",  
        SUM("SalesQuantity") AS "TotalSalesQuantity",
        SUM("SalesDollars") AS "TotalSalesDollars",
        SUM("SalesPrice") AS "TotalSalesPrice",
        SUM("ExciseTax") AS "TotalExciseTax"
    FROM sales
    GROUP BY "VendorNo", "Brand"
)

SELECT
    ps."VendorNumber",
    ps."VendorName",
    ps."Brand",
    ps."Description",
    ps."ActualPrice",
    ps."PurchasePrice",
    ps."Volume",
    ps."TotalPurchaseQuantity",
    ps."TotalPurchaseDollars",
    ss."TotalSalesQuantity",
    ss."TotalSalesDollars",
    ss."TotalSalesPrice",
    ss."TotalExciseTax",
    fs."FreightCost"
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps."VendorNumber" = ss."VendorNo"
    AND ps."Brand" = ss."Brand"
LEFT JOIN FreightSummary fs
    ON ps."VendorNumber" = fs."VendorNumber"
ORDER BY ps."TotalPurchaseDollars" DESC
""", engine)

end = time.time()
print(f"Query execution time: {end - start:.2f} seconds")
vendor_sales_summary

Query execution time: 7.72 seconds


,VendorNumber,VendorName,Brand,Description,ActualPrice,PurchasePrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,36.99,26.27,1750,145080.0,3811251.60,142049.0,5101919.51,672819.31,260999.20,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,28.99,23.19,1750,164038.0,3804041.22,160247.0,4819073.49,561512.37,294438.66,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,24.99,18.24,1750,187407.0,3418303.68,187140.0,4538120.60,461140.15,343854.07,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,22.99,16.17,1750,201682.0,3261197.94,200412.0,4475972.88,420050.01,368242.80,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,29.99,21.89,1750,138109.0,3023206.01,135838.0,4223107.62,545778.28,249587.83,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,4.99,1.32,750,2.0,2.64,5.0,15.95,10.96,0.55,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.49,0.39,50,6.0,2.34,134.0,65.66,1.47,7.04,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.99,0.74,50,2.0,1.48,2.0,1.98,0.99,0.10,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.99,1.47,200,1.0,1.47,72.0,143.28,77.61,15.12,257032.07


In [9]:
vendor_sales_summary.dtypes

VendorNumber               int64
VendorName                object
Brand                      int64
Description               object
ActualPrice              float64
PurchasePrice            float64
Volume                    object
TotalPurchaseQuantity    float64
TotalPurchaseDollars     float64
TotalSalesQuantity       float64
TotalSalesDollars        float64
TotalSalesPrice          float64
TotalExciseTax           float64
FreightCost              float64
dtype: object

In [10]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype("float32")
vendor_sales_summary.dtypes

VendorNumber               int64
VendorName                object
Brand                      int64
Description               object
ActualPrice              float64
PurchasePrice            float64
Volume                   float32
TotalPurchaseQuantity    float64
TotalPurchaseDollars     float64
TotalSalesQuantity       float64
TotalSalesDollars        float64
TotalSalesPrice          float64
TotalExciseTax           float64
FreightCost              float64
dtype: object

In [11]:
vendor_sales_summary.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
Description                0
ActualPrice                0
PurchasePrice              0
Volume                     0
TotalPurchaseQuantity      0
TotalPurchaseDollars       0
TotalSalesQuantity       178
TotalSalesDollars        178
TotalSalesPrice          178
TotalExciseTax           178
FreightCost                0
dtype: int64

In [12]:
vendor_sales_summary.fillna(0, inplace=True)

In [13]:
vendor_sales_summary.isnull().sum()

VendorNumber             0
VendorName               0
Brand                    0
Description              0
ActualPrice              0
PurchasePrice            0
Volume                   0
TotalPurchaseQuantity    0
TotalPurchaseDollars     0
TotalSalesQuantity       0
TotalSalesDollars        0
TotalSalesPrice          0
TotalExciseTax           0
FreightCost              0
dtype: int64

In [14]:
vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP          ', 'MARTIGNETTI COMPANIES',
       'PERNOD RICARD USA          ', 'DIAGEO NORTH AMERICA INC   ',
       'BACARDI USA INC            ', 'JIM BEAM BRANDS COMPANY    ',
       'MAJESTIC FINE WINES        ', 'ULTRA BEVERAGE COMPANY LLP ',
       'STOLI GROUP,(USA) LLC      ', 'PROXIMO SPIRITS INC.       ',
       'MOET HENNESSY USA INC      ', 'CAMPARI AMERICA            ',
       'SAZERAC CO INC             ', 'CONSTELLATION BRANDS INC   ',
       'M S WALKER INC             ', 'SAZERAC NORTH AMERICA INC. ',
       'PALM BAY INTERNATIONAL INC ', 'REMY COINTREAU USA INC     ',
       'SIDNEY FRANK IMPORTING CO  ', 'E & J GALLO WINERY         ',
       'WILLIAM GRANT & SONS INC   ', 'HEAVEN HILL DISTILLERIES   ',
       'DISARONNO INTERNATIONAL LLC', 'EDRINGTON AMERICAS         ',
       'CASTLE BRANDS CORP.        ', 'SOUTHERN WINE & SPIRITS NE ',
       'STE MICHELLE WINE ESTATES  ', 'TRINCHERO FAMILY ESTATES   ',
       'MHW LTD                    ', 'W

In [15]:
vendor_sales_summary['VendorName']=vendor_sales_summary['VendorName'].str.strip()

In [16]:
vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP', 'MARTIGNETTI COMPANIES', 'PERNOD RICARD USA',
       'DIAGEO NORTH AMERICA INC', 'BACARDI USA INC',
       'JIM BEAM BRANDS COMPANY', 'MAJESTIC FINE WINES',
       'ULTRA BEVERAGE COMPANY LLP', 'STOLI GROUP,(USA) LLC',
       'PROXIMO SPIRITS INC.', 'MOET HENNESSY USA INC', 'CAMPARI AMERICA',
       'SAZERAC CO INC', 'CONSTELLATION BRANDS INC', 'M S WALKER INC',
       'SAZERAC NORTH AMERICA INC.', 'PALM BAY INTERNATIONAL INC',
       'REMY COINTREAU USA INC', 'SIDNEY FRANK IMPORTING CO',
       'E & J GALLO WINERY', 'WILLIAM GRANT & SONS INC',
       'HEAVEN HILL DISTILLERIES', 'DISARONNO INTERNATIONAL LLC',
       'EDRINGTON AMERICAS', 'CASTLE BRANDS CORP.',
       'SOUTHERN WINE & SPIRITS NE', 'STE MICHELLE WINE ESTATES',
       'TRINCHERO FAMILY ESTATES', 'MHW LTD', 'WINE GROUP INC',
       'PERFECTA WINES', 'LUXCO INC', 'TREASURY WINE ESTATES',
       'DIAGEO CHATEAU ESTATE WINES', 'SHAW ROSS INT L IMP LTD',
       'PINE STATE TRADING CO', 'PHILLIPS PRO

In [17]:
vendor_sales_summary['GrossProfit']=vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [18]:
vendor_sales_summary['ProfitMargin']=(vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalSalesDollars'])*100

In [19]:
vendor_sales_summary['StockTurnover']=vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']

In [20]:
vendor_sales_summary['SalesPurchaseRatio']=vendor_sales_summary['TotalSalesDollars']/vendor_sales_summary['TotalPurchaseDollars']

In [21]:
# Obtain a raw DBAPI connection from the SQLAlchemy engine
raw_connection = engine.raw_connection()

# Create a cursor from the raw connection
cursor = raw_connection.cursor()

# Example: Execute a raw SQL command
cursor.execute("SELECT version();")
result = cursor.fetchone()
print(result)

# Close the cursor and connection when done
cursor.close()
raw_connection.close()


('PostgreSQL 17.5 on x86_64-windows, compiled by msvc-19.44.35209, 64-bit',)


In [22]:
import psycopg2
conn = psycopg2.connect(
        host=PG_HOST,
        database=PG_DATABASE,
        user=PG_USER,
        password=PG_PASSWORD,
        port=PG_PORT
    )

In [23]:
cursor=conn.cursor()

In [24]:
vendor_sales_summary.columns

Index(['VendorNumber', 'VendorName', 'Brand', 'Description', 'ActualPrice',
       'PurchasePrice', 'Volume', 'TotalPurchaseQuantity',
       'TotalPurchaseDollars', 'TotalSalesQuantity', 'TotalSalesDollars',
       'TotalSalesPrice', 'TotalExciseTax', 'FreightCost', 'GrossProfit',
       'ProfitMargin', 'StockTurnover', 'SalesPurchaseRatio'],
      dtype='object')

In [25]:
conn.rollback()

In [26]:
import psycopg2

# Connection parameters
PG_HOST = "localhost"
PG_PORT = "5432"
PG_USER = "postgres"
PG_PASSWORD = "Pass@123"
PG_DATABASE = "inventory"

try:
    # Connect to database
    conn = psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        user=PG_USER,
        password=PG_PASSWORD,
        database=PG_DATABASE
    )

    cursor = conn.cursor()

    # Rollback any previous failed transaction
    conn.rollback()

    # Now run your create table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS vendor_sales_summary (
            VendorNumber INT,
            VendorName VARCHAR(100),
            Brand INT,
            Description VARCHAR(100),
            ActualPrice DECIMAL(10,2),
            PurchasePrice DECIMAL(10,2),
            Volume DECIMAL(10,2),
            TotalPurchaseQuantity INT,
            TotalPurchaseDollars DECIMAL(15,2),
            TotalSalesQuantity INT,
            TotalSalesDollars DECIMAL(15,2),
            TotalSalesPrice DECIMAL(15,2),
            TotalExciseTax DECIMAL(15,2),
            FreightCost DECIMAL(15,2),
            GrossProfit DECIMAL(15,2),
            ProfitMargin DECIMAL(15,2),
            StockTurnover DECIMAL(15,2),
            SalesPurchaseRatio DECIMAL(15,2),
            PRIMARY KEY (VendorNumber, Brand)
        );
    """)
    
    conn.commit()
    print("Table created successfully.")

    cursor.close()
    conn.close()

except Exception as e:
    print("Error:", e)
    conn.rollback()
    if cursor: cursor.close()
    if conn: conn.close()


Table created successfully.


In [27]:
pd.read_sql_query("select * from vendor_sales_summary " ,engine)


,VendorNumber,VendorName,Brand,Description,ActualPrice,PurchasePrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnover,SalesPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,36.99,26.27,1750.0,145080.0,3811251.60,142049.0,5101919.51,672819.31,260999.20,68601.68,1290667.91,25.297693,0.979108,1.338647
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,28.99,23.19,1750.0,164038.0,3804041.22,160247.0,4819073.49,561512.37,294438.66,144929.24,1015032.27,21.062810,0.976890,1.266830
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,24.99,18.24,1750.0,187407.0,3418303.68,187140.0,4538120.60,461140.15,343854.07,123780.22,1119816.92,24.675786,0.998575,1.327594
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,22.99,16.17,1750.0,201682.0,3261197.94,200412.0,4475972.88,420050.01,368242.80,257032.07,1214774.94,27.139908,0.993703,1.372493
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,29.99,21.89,1750.0,138109.0,3023206.01,135838.0,4223107.62,545778.28,249587.83,257032.07,1199901.61,28.412764,0.983556,1.396897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,4.99,1.32,750.0,2.0,2.64,5.0,15.95,10.96,0.55,27100.41,13.31,83.448276,2.500000,6.041667
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.49,0.39,50.0,6.0,2.34,134.0,65.66,1.47,7.04,50293.62,63.32,96.436186,22.333333,28.059829
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.99,0.74,50.0,2.0,1.48,2.0,1.98,0.99,0.10,14069.87,0.50,25.252525,1.000000,1.337838
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.99,1.47,200.0,1.0,1.47,72.0,143.28,77.61,15.12,257032.07,141.81,98.974037,72.000000,97.469388


In [28]:
vendor_sales_summary.to_sql( 'vendor_sales_summary' ,engine,if_exists ='replace', index=False)

692